In [1]:
def iso8601_to_timedelta(iso8601_duration):
    import re
    from datetime import timedelta  # 追加

    # ISO 8601形式の文字列から時間、分、秒を取得
    duration_parts = re.findall(r'(\d+)([HMS])', iso8601_duration)
    hours, minutes, seconds = 0, 0, 0

    for value, unit in duration_parts:
        if unit == 'H':
            hours = int(value)
        elif unit == 'M':
            minutes = int(value)
        elif unit == 'S':
            seconds = int(value)

    # timedelta型に変換
    duration = timedelta(hours=hours, minutes=minutes, seconds=seconds)

    return duration

In [2]:
def skip_key_chapter(v_id):
    #pip install google-api-python-clientをターミナルで実行
    import googleapiclient.discovery
    from googleapiclient.discovery import build
    import requests
    import time
    import os
    import re
    import webbrowser
    import datetime
    from datetime import datetime, timedelta
    import subprocess
    import psutil
#     import pyautogui
    from selenium import webdriver
    from selenium.webdriver.chrome import service
    from config import api_key

    # 利用するAPIサービス
    YOUTUBE_API_SERVICE_NAME = 'youtube'
    YOUTUBE_API_VERSION = 'v3'

    # APIキーを指定
    YOUTUBE_API_KEY = f"{api_key}"

    # API のビルドと初期化
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=YOUTUBE_API_KEY)

    # 動画IDを指定
    video_id = v_id
    
    # 動画のURLを生成
    video_url = f'https://www.youtube.com/watch?v={video_id}&t={0}s'

    # 動画情報を取得
    video_response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()

    # 動画の説明文を取得
    video_description = video_response['items'][0]['snippet']['description']
    
    # 動画の詳細情報を取得
    video_response2 = youtube.videos().list(
        part='contentDetails',
        id=video_id
    ).execute()
    # 動画の総時間を取得
    duration = video_response2['items'][0]['contentDetails']['duration']
    # 再生時間durationをISO8601形式からdatetime型に直す.
    duration = iso8601_to_timedelta(duration)

    # カレントディレクトリを保存
    # wdを設定してcmdを使うときは必要
#     cwd = os.getcwd()
    
    # print(f"動画の総時間: {duration}")

    # キーワードを指定(※動画内で2回以上出現するものは不可)
    keyword = 'シャンプー'

    # 説明文の解析しタイムスタンプのリストtimestampsを取得
    timestamps = re.findall(r'\d+:\d+:\d+', video_description)
    # MM:SS型
    if not timestamps:
         timestamps = re.findall(r'\d+:\d+', video_description)
    print(timestamps)
    
    # キーワードが見つかったかどうかを示すフラグ
    keyword_found = False

    # タイムスタンプとキーワードを含む行を抽出
    pattern = rf'(\d+:\d+:\d+).*?{keyword}.*?$'
    matches = re.findall(pattern, video_description, re.MULTILINE)
    # mathesが空でないならばkeyword_foundをTrueにする
    if matches: 
        keyword_found = True
        print("keyword was found")
    # タイムスタンプが"MM:SS"形式の場合
#     else:
#         pattern = rf'(\d+:\d+).*?{keyword}.*?$'
#         matches = re.findall(pattern, video_description, re.MULTILINE)
#         if matches:
#             keyword_found = True
#             print("keyword was found")
        
    # ドライバーを設定
    driver = webdriver.Chrome(options=get_chrome_options())
    
    if not keyword_found:
        # durationは先ほどiso8601_to_timedelta()でtime_delta型に直しているのでint型に丸めるだけでよい.
        duration_sec = int(duration.total_seconds())
        print(duration_sec)
        # ブラウザで動画を再生
        driver.get(video_url)
        #webbrowser.get('"C:\Program Files\Google\Chrome\Application\chrome.exe" %s &').open(video_url)
        # 終了時間まで待機
        # time.sleep(5)
        time.sleep(5+duration_sec)
        # プロセス終了
        driver.quit()
        #subprocess.Popen(["taskkill", "/F", "/IM", "chrome.exe"])
        return
    
    else:
        # 抽出したタイムスタンプを表示
        for match in matches:
            print(match)

       
        matching_timestamps = []
        index = 0
        # タイムスタンプとキーワードが一致する行を抽出
        for match in matches:
            for timestamp in timestamps:
                if timestamp in match:
                    # キーワードを含むタイムスタンプのインデックス(番目)を取得
                    index = timestamps.index(timestamp)
                    print(index)
                    # 一致したタイムスタンプを取得(キーワードを含むタイムスタンプ)
                    matching_timestamp = timestamps[index]
                    # matching_timestamps.append(timestamp) (リストに格納するやり方)
                    # 次のチャプターのタイムスタンプを取得
                    next_timestamp = timestamps[index+1]

        # 一致するタイムスタンプを表示
        print(f"Maching timestamp: {timestamps[index]}")
        # print(f"Maching timestamp: {matching_timestamp}") でもよい
        # リストにぶち込んだ場合は以下で表示
        # for timestamp in matching_timestamps:
        #     print(f"Matching timestamp: {timestamp}")    
        print(f"Next timestamp: {timestamps[index+1]}")
        
        # 説明文のデバッグ出力
        # print(video_description)

        # 時間文字列をdatetimeオブジェクトに変換
#         matching_timestamp = datetime.strptime(matching_timestamp, "%M:%S")
#         next_timestamp = datetime.strptime(next_timestamp, "%M:%S")
#         base_time = datetime.strptime('00:00', "%M:%S")
        matching_timestamp = datetime.strptime(matching_timestamp, "%H:%M:%S")
        next_timestamp = datetime.strptime(next_timestamp, "%H:%M:%S")
        base_time = datetime.strptime('0:00:00', "%H:%M:%S")

        # total_seconds()メゾットを使うため引き算してtimedelta型にする.
        matching_timestamp =  matching_timestamp - base_time
        next_timestamp = next_timestamp - base_time
        remaining_time = duration - next_timestamp
        # duration = duration - base_time

        # 秒数に変換
        matching_timestamp_sec = int(matching_timestamp.total_seconds())
        next_timestamp_sec = int(next_timestamp.total_seconds())
        remaining_time_sec = int(remaining_time.total_seconds())
        # duration_sec = int(duration.total_seconds())
        print(f"next_timestamp_sec: {next_timestamp_sec}")
        print(f"remaining_time_sec: {remaining_time_sec}")

        # ブラウザで動画を開く
        driver.get(video_url)
        #webbrowser.get('"C:\Program Files\Google\Chrome\Application\chrome.exe" %s &').open(video_url)
        
        # キーワードが出現するチャプター開始時刻まで待機(接続時間: 5s)
        time.sleep(matching_timestamp_sec)
        # キーワードを含むチャプター開始位置でChromeをタスクキル
        driver.quit()
        #subprocess.Popen(["taskkill", "/F", "/IM", "chrome.exe"])
        # 再生位置を次のチャプターの開始時刻位置に変更した動画urlを取得
        skipped_video_url = f'https://www.youtube.com/watch?v={video_id}&t={next_timestamp_sec}s'
        # 必要な待機時間(競合回避)
        time.sleep(1)  
        # ドライバを再設定(Max Retry Errorを避けるため)
        driver = webdriver.Chrome(options=get_chrome_options())
        # ブラウザでスキップされた動画を開く
        driver.get(skipped_video_url)
#         webbrowser.get('"C:\Program Files\Google\Chrome\Application\chrome.exe" %s &').open(skipped_video_url)
        #スキップ位置から終了時刻まで待機
        time.sleep(remaining_time_sec)
        # 動画終了時刻でChromeをタスクキル(動画終了)
        driver.quit()
#         subprocess.Popen(["taskkill", "/F", "/IM", "chrome.exe"])

In [3]:
# URLリストからYouTubeの動画id取り出し
# 通常URLと短縮URLに対応。未対応URLではエラーメッセージ表示
# 引数：URLのリスト
# 戻り値：抽出できた動画idのリスト
## 出典: https://qiita.com/yukuduri/items/3054c28f50547cf7bec1 ##
def pick_up_vid_list(url_list):
    import urllib.parse
    import re
    vid_list = []
    pattern_watch = 'https://www.youtube.com/watch?'
    pattern_short = 'https://youtu.be/'
    pattern_live = 'https://www.youtube.com/live/'

    for i, url in enumerate(url_list):
        # 通常URLのとき
        if re.match(pattern_watch,url):
            yturl_qs = urllib.parse.urlparse(url).query
            vid = urllib.parse.parse_qs(yturl_qs)['v'][0]
            vid_list.append(vid)

        # 短縮URLのとき
        elif re.match(pattern_short,url):
          # "https://youtu.be/"に続く11文字が動画ID
            vid = url[17:28]
            vid_list.append(vid)
        # ライブ配信URLのとき
        elif re.match(pattern_live,url):
          # "https://www.youtube.com/live"に続く11文字が動画ID
            vid = url[29:40]
            vid_list.append(vid)

        else:
            print('error:\n  URLは\"https://www.youtube.com/watch?\"か')
            print('\"https://www.youtube.com/live\"か')
            print('  \"https://youtu.be/\"で始まるURLを指定してください。')
            print('  - '+ str(i+1)+ '個目：' + url)
    return vid_list

In [4]:
# webdriver.Chromeのchrome_optionのリスト
def get_chrome_options():
    # import getpass
    from selenium import webdriver
    from selenium.webdriver.chrome import service
    import time
    chrome_driver_path = 'C:\WebDriver\bin\chromedriver.exe'  # ChromeDriverのパスを指定
    driver = webdriver.Chrome(chrome_driver_path)

    # 以下のオプションはhttps://qiita.com/kawagoe6884/items/cea239681bdcffe31828のものをコピペしてます.
    from selenium.webdriver.chrome.options import Options
    options = Options()
    #options.add_argument('--blink-settings=imagesEnabled=false')                    # 画像の非表示
    options.add_argument('--disable-blink-features=AutomationControlled')           # navigator.webdriver=false とする設定
    options.add_argument('--disable-browser-side-navigation')                       # Timed out receiving message from renderer: の修正
    options.add_argument('--disable-dev-shm-usage')                                 # ディスクのメモリスペースを使う
    options.add_argument('--disable-extensions')                                    # すべての拡張機能を無効
    options.add_argument('--disable-gpu')                                           # GPUハードウェアアクセラレーションを無効
    # options.add_argument('--headless')                                            # ヘッドレスモードで起動
    options.add_argument('--ignore-certificate-errors')                             # SSL認証(この接続ではプライバシーが保護されません)を無効
    options.add_argument('--incognito')                                             # シークレットモードで起動
    options.add_argument('--no-sandbox')                                            # Chromeの保護機能を無効
    options.add_argument('--start-maximized')                                     # 初期のウィンドウサイズを最大化
    options.add_argument('--window-size=1920,1080')                                 # 初期のウィンドウサイズを指定
    options.add_experimental_option("excludeSwitches", ['enable-automation'])       # Chromeは自動テスト ソフトウェア~~ を非表示
    options.add_experimental_option("excludeSwitches", ['enable-logging'])          # ERROR:device_event_log_impl.cc~~ などのログ出力を無効化
    options.add_experimental_option('useAutomationExtension', False)                # 拡張機能の自動更新を停止
    options.add_argument('--autoplay-policy=no-user-gesture-required')              # 自動再生を有効化
    prefs = {
        'profile.default_content_setting_values.notifications' : 2,                 # 通知ポップアップを無効
        'credentials_enable_service' : False,                                       # パスワード保存のポップアップを無効
        'profile.password_manager_enabled' : False,                                 # パスワード保存のポップアップを無効
        # 'download.default_directory' : download_dir                               # ダウンロード先のディレクトリを指定
    }
    options.add_experimental_option('prefs', prefs)
    # テスト用Chromeの実行ファイルへのパス
    options.binary_location = "C:\Program Files\chrome-win64\chrome.exe"
    return options

In [5]:
# main
import pandas as pd
import numpy 

if __name__ == "__main__":
    df = pd.read_excel('vurl_list.xlsx', usecols=['URL']).squeeze('columns')
    url_list = df.to_numpy().tolist()
    print(url_list)
    vid_list = pick_up_vid_list(url_list)
    print(vid_list)
    chrome_driver_path = 'C:\WebDriver\bin\chromedriver.exe'  # ChromeDriverのパスを指定
    options = get_chrome_options()
    for vid in enumerate(vid_list):
        #vid[0]はインデックス, vid[1]に動画IDが格納されている.
        print(vid[1])
        skip_key_chapter(vid[1])

['https://www.youtube.com/watch?v=2VpvV_7xvgM']
['2VpvV_7xvgM']


AttributeError: 'str' object has no attribute 'capabilities'

In [ ]:
#以下メモ
import urllib.parse
import re
url='https://www.youtube.com/watch?v=UTEfAA5PSJg'
yturl_qs = urllib.parse.urlparse(url).query
vid = urllib.parse.parse_qs(yturl_qs)['v'][0]
print(yturl_qs)
print(vid)
print(urllib.parse.parse_qs(yturl_qs))

In [ ]:
print(vid_list)

In [ ]:
import subprocess
import time
#プロセスID(pid)を取得するのに必要.
import psutil
import signal
import os

# Chrome プロセスを取得
# chrome_processes = [p for p in psutil.process_iter(attrs=['pid', 'name']) if 'chrome.exe' in p.info['name']]
    
# Chrome プロセスを終了
# for process in chrome_processes:
#     try:
#         process.terminate()  # プロセスを終了
#     except psutil.NoSuchProcess:
#         pass

# os.chdir("C:\\Users\\yuki\\OneDrive\\デスクトップ")
# p = subprocess.Popen("chrome_kill.bat")
# stdout, stderr = p.communicate()

# Chrome プロセスを終了
# os.chdir("C:\\Users\\yuki\\OneDrive\\デスクトップ")
# os.startfile("chrome_kill.bat")

# # Chrome プロセスを終了
# subprocess.run(["taskkill", "/F", "/IM", "chrome.exe"])


# video_url='\"https://www.youtube.com/watch?v=UTEfAA5PSJg\"'
# cmd = f"start chrome.exe {video_url}"

# cmd2 = ["start", "chrome.exe", "\"https://www.youtube.com/watch?v=UTEfAA5PSJg\""]
# p = subprocess.Popen(cmd2)
# time.sleep(5)

# # プロセスのPIDを取得
# pid = p.pid 

# # PIDを使用してプロセスを終了
# process = psutil.Process(pid)
# process.terminate()

# os.system("taskkill /im chrome.exe /f")

#         os.chdir("C:\\Users\\yuki\\OneDrive\\デスクトップ")
#         p = subprocess.Popen("chrome_kill.bat")
#         stdout, stderr = p.communicate()
#         os.chdir(cwd)

# subprocess.run(["powershell", "-command", "$wshell = New-Object -ComObject wscript.shell; \
#                         $wshell.AppActivate('Google Chrome'); Start-Sleep -Seconds 1; $wshell.SendKeys('% r')"])

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyautogui
from time import sleep

# import webbrowser

# #ChromeDriverのパスを変数に設定
# CHROMEDRIVER = "%Path%\\chromedriver.exe"
# #ChromeDriverのstartとstopを制御するServiceオブジェクトを介してパスを渡す
# chrome_service = service.Service(executable_path=CHROMEDRIVER)
# #Chromeを起動
# driver = webdriver.Chrome(service=chrome_service)
# #指定したURLに遷移する
# driver.get("https://www.google.co.jp")
# #検索テキストボックスの要素をname属性名から取得
# element = driver.find_element(By.NAME, "q")
# #検索テキストボックスでシフトボタンを押下しながら"selenium"を入力
# element.send_keys(Keys.SHIFT,"selenium")
# #カーソルを１文字分左にずらす
# element.send_keys(Keys.ARROW_LEFT)
# #BackSpaceを１回押下し１文字分消去
# element.send_keys(Keys.BACK_SPACE)
# #Enterキーを押下する
# element.send_keys(Keys.ENTER)

# # Alt+Tabを押す
# pyautogui.hotkey('win', '2')

 
#ChromeDriverのパスを変数に設定
# CHROMEDRIVER = "D:\driver\chromedriver.exe"
# #ChromeDriverのstartとstopを制御するServiceオブジェクトを介してパスを渡す
# chrome_service = service.Service(executable_path=CHROMEDRIVER)
# #Chromeを起動

# from selenium.webdriver.chrome.options import Options

# # options = Options();
# # options.add_argument('--no-sandbox')
# # options.add_argument('--headless')
# # options.binary_location = '/usr/bin/chromium-browser'
# driver = webdriver.Chrome()
# # driver = webdriver.Chrome()
# #指定したURLに遷移
# driver.get("https://www.google.co.jp")
# #カレントウインドウを最大化する
# driver.maximize_window()
# video_url='https://www.youtube.com/watch?v=UTEfAA5PSJg'
# webbrowser.get('"C:\Program Files\Google\Chrome\Application\chrome.exe" %s &').open_new(video_url)

 
#ChromeDriverのパスを変数に設定
#CHROMEDRIVER = "C:\\WebDriver\\bin\\chromedriver.exe"
#ChromeDriverのstartとstopを制御するServiceオブジェクトを介してパスを渡す
#chrome_service = service.Service(executable_path=CHROMEDRIVER)
#Chromeを起動
#driver = webdriver.Chrome(service=chrome_service)

# Chrome のオプションを設定する
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')

# Selenium Server に接続する
# driver = webdriver.Remote(
#     command_executor='http://localhost:8888/notebooks/Documents/YouTube_API',
#     desired_capabilities=options.to_capabilities(),
#     options=options,
# )

# browser = webdriver.Chrome('chromedriver.exe')




# options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # ヘッドレスモードを有効にする
# driver = webdriver.Chrome(options=options)

# Chrome Optionsの設定
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument('--headless')                 # headlessモードを使用する
# options.add_argument('--disable-gpu')              # headlessモードで暫定的に必要なフラグ(そのうち不要になる)
# options.add_argument('--disable-extensions')       # すべての拡張機能を無効にする。ユーザースクリプトも無効にする
# options.add_argument('--proxy-server="direct://"') # Proxy経由ではなく直接接続する
# options.add_argument('--proxy-bypass-list=*')      # すべてのホスト名
# options.add_argument('--start-maximized')          # 起動時にウィンドウを最大化する

# opt = webdriver.ChromeOptions()
# opt.binary_location("/path/to/other/chrome/binary")

# driver = webdriver.Chrome() #ウェブドライバのパスの設定しておく必要あり
# driver.get("https://www.youtube.com/")
# time.sleep(5)
# driver.close()

# driver = webdriver.Chrome()
# #指定したURLに遷移
# driver.implicitly_wait(1)
# driver.get("https://www.google.co.jp")
# #カレントウインドウを最大化する
# driver.maximize_window()

#from selenium import webdriver
# import chromedriver_binary
# driver = webdriver.Chrome()
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument('--headless')
# options.add_argument("--no-sandbox")

# print('before')
# driver = webdriver.Chrome(chrome_options=options)
# print('after')
# driver.get('https://www.google.co.jp/')

# Win+upを押してChromeをアクティブウィンドウにする.
#         pyautogui.hotkey('win', 'up')